In [1]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import re
import numpy as np
import time
import matplotlib as plt
import dateutil.parser
import random
import datetime

In [2]:
top1000 = pd.read_pickle("top1000.pkl")  
top1000.head(50)

,Title,Domestic_Gr,Dom_Open,Budget,Rel_Date,Runtime,Genre,Rating,Distributor,Cast,Director
0,Star Wars: Episode VII - The Force Awakens (2015),"$936,662,225","$247,966,675","$245,000,000","December 16, 2015\n (EMEA, APAC)",2 hr 18 min,Action\n \n Adventure\n \n ...,PG-13,Walt Disney Studios Motion PicturesSee full co...,Daisy Ridley\n \n ReySee more,J.J. Abrams\n \n Director
1,Avengers: Endgame (2019),"$858,373,000","$357,115,007","$356,000,000","April 24, 2019\n (21 markets)",3 hr 1 min,Action\n \n Adventure\n \n ...,PG-13,Walt Disney Studios Motion PicturesSee full co...,Robert Downey Jr.\n \n Tony Stark/Ir...,Anthony Russo\n \n Director
2,Spider-Man: No Way Home (2021),"$804,761,122","$260,138,569",NaN,"December 15, 2021\n (EMEA, APAC)",2 hr 28 min,Action\n \n Adventure\n \n ...,PG-13,Sony Pictures Entertainment (SPE)See full comp...,Tom Holland,Jon Watts
3,Avatar (2009),"$760,507,625","$77,025,481","$237,000,000","December 16, 2009\n (LATAM, EMEA)",2 hr 42 min,Action\n \n Adventure\n \n ...,PG-13,Twentieth Century FoxSee full company informat...,Sam Worthington\n \n Jake SullySee more,James Cameron\n \n Director
4,Black Panther (2018),"$700,426,566","$202,003,951",NaN,"February 13, 2018\n (APAC, EMEA)",2 hr 14 min,Action\n \n Adventure\n \n ...,PG-13,Walt Disney Studios Motion PicturesSee full co...,Chadwick Boseman\n \n T'Challa/Black...,Ryan Coogler\n \n Director
5,Avengers: Infinity War (2018),"$678,815,482","$257,698,183",NaN,"April 25, 2018\n (APAC, EMEA)",2 hr 29 min,Action\n \n Adventure\n \n ...,PG-13,Walt Disney Studios Motion PicturesSee full co...,Robert Downey Jr.\n \n Tony Stark/Ir...,Anthony Russo\n \n Director
6,Titanic (1997),"$659,363,944","$28,638,131","$200,000,000","December 19, 1997\n (Domestic)",3 hr 14 min,Drama\n \n Romance,PG-13,Paramount PicturesSee full company information...,Leonardo DiCaprio\n \n Jack DawsonSe...,James Cameron\n \n Director
7,Jurassic World (2015),"$653,406,625","$208,806,270","$150,000,000","June 10, 2015\n (9 markets)",2 hr 4 min,Action\n \n Adventure\n \n ...,PG-13,Universal PicturesSee full company information...,Chris Pratt\n \n Owen GradySee more,Colin Trevorrow\n \n Director
8,The Avengers (2012),"$623,357,910","$207,438,708","$220,000,000","April 25, 2012\n (APAC, EMEA)",2 hr 23 min,Action\n \n Adventure\n \n ...,PG-13,Walt Disney Studios Motion PicturesSee full co...,Robert Downey Jr.\n \n Tony Stark/Ir...,Joss Whedon\n \n Director
9,Star Wars: Episode VIII - The Last Jedi (2017),"$620,181,382","$220,009,584","$317,000,000","December 13, 2017\n (APAC, EMEA)",2 hr 32 min,Action\n \n Adventure\n \n ...,PG-13,Walt Disney Studios Motion PicturesSee full co...,Daisy Ridley\n \n ReySee more,Rian Johnson\n \n Director


In [3]:
top1000.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 0 to 999
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Title        1000 non-null   object
 1   Domestic_Gr  1000 non-null   object
 2   Dom_Open     969 non-null    object
 3   Budget       807 non-null    object
 4   Rel_Date     1000 non-null   object
 5   Runtime      1000 non-null   object
 6   Genre        1000 non-null   object
 7   Rating       1000 non-null   object
 8   Distributor  1000 non-null   object
 9   Cast         1000 non-null   object
 10  Director     1000 non-null   object
dtypes: object(11)
memory usage: 93.8+ KB


In [4]:
#there is a lot of missing values. Will have to assess later.
#cleaning Target variable: Domestic Gross Value
top1000["Domestic_Gr"]=top1000["Domestic_Gr"].str.replace('$','').str.replace(',', '')
top1000.head()

/var/folders/5b/gqr92vkj3k37f4bn6sh6wl6w0000gn/T/ipykernel_75510/2074297602.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  top1000["Domestic_Gr"]=top1000["Domestic_Gr"].str.replace('$','').str.replace(',', '')


,Title,Domestic_Gr,Dom_Open,Budget,Rel_Date,Runtime,Genre,Rating,Distributor,Cast,Director
0,Star Wars: Episode VII - The Force Awakens (2015),936662225,"$247,966,675","$245,000,000","December 16, 2015\n (EMEA, APAC)",2 hr 18 min,Action\n \n Adventure\n \n ...,PG-13,Walt Disney Studios Motion PicturesSee full co...,Daisy Ridley\n \n ReySee more,J.J. Abrams\n \n Director
1,Avengers: Endgame (2019),858373000,"$357,115,007","$356,000,000","April 24, 2019\n (21 markets)",3 hr 1 min,Action\n \n Adventure\n \n ...,PG-13,Walt Disney Studios Motion PicturesSee full co...,Robert Downey Jr.\n \n Tony Stark/Ir...,Anthony Russo\n \n Director
2,Spider-Man: No Way Home (2021),804761122,"$260,138,569",NaN,"December 15, 2021\n (EMEA, APAC)",2 hr 28 min,Action\n \n Adventure\n \n ...,PG-13,Sony Pictures Entertainment (SPE)See full comp...,Tom Holland,Jon Watts
3,Avatar (2009),760507625,"$77,025,481","$237,000,000","December 16, 2009\n (LATAM, EMEA)",2 hr 42 min,Action\n \n Adventure\n \n ...,PG-13,Twentieth Century FoxSee full company informat...,Sam Worthington\n \n Jake SullySee more,James Cameron\n \n Director
4,Black Panther (2018),700426566,"$202,003,951",NaN,"February 13, 2018\n (APAC, EMEA)",2 hr 14 min,Action\n \n Adventure\n \n ...,PG-13,Walt Disney Studios Motion PicturesSee full co...,Chadwick Boseman\n \n T'Challa/Black...,Ryan Coogler\n \n Director


In [5]:
#cleaning Feature variables: Dom_Open, Budget
top1000["Dom_Open"]=top1000["Dom_Open"].str.replace('$','').str.replace(',', '')
top1000["Budget"]=top1000["Budget"].str.replace('$','').str.replace(',', '')
top1000.head()

/var/folders/5b/gqr92vkj3k37f4bn6sh6wl6w0000gn/T/ipykernel_75510/1441856404.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  top1000["Dom_Open"]=top1000["Dom_Open"].str.replace('$','').str.replace(',', '')
/var/folders/5b/gqr92vkj3k37f4bn6sh6wl6w0000gn/T/ipykernel_75510/1441856404.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  top1000["Budget"]=top1000["Budget"].str.replace('$','').str.replace(',', '')


,Title,Domestic_Gr,Dom_Open,Budget,Rel_Date,Runtime,Genre,Rating,Distributor,Cast,Director
0,Star Wars: Episode VII - The Force Awakens (2015),936662225,247966675,245000000,"December 16, 2015\n (EMEA, APAC)",2 hr 18 min,Action\n \n Adventure\n \n ...,PG-13,Walt Disney Studios Motion PicturesSee full co...,Daisy Ridley\n \n ReySee more,J.J. Abrams\n \n Director
1,Avengers: Endgame (2019),858373000,357115007,356000000,"April 24, 2019\n (21 markets)",3 hr 1 min,Action\n \n Adventure\n \n ...,PG-13,Walt Disney Studios Motion PicturesSee full co...,Robert Downey Jr.\n \n Tony Stark/Ir...,Anthony Russo\n \n Director
2,Spider-Man: No Way Home (2021),804761122,260138569,NaN,"December 15, 2021\n (EMEA, APAC)",2 hr 28 min,Action\n \n Adventure\n \n ...,PG-13,Sony Pictures Entertainment (SPE)See full comp...,Tom Holland,Jon Watts
3,Avatar (2009),760507625,77025481,237000000,"December 16, 2009\n (LATAM, EMEA)",2 hr 42 min,Action\n \n Adventure\n \n ...,PG-13,Twentieth Century FoxSee full company informat...,Sam Worthington\n \n Jake SullySee more,James Cameron\n \n Director
4,Black Panther (2018),700426566,202003951,NaN,"February 13, 2018\n (APAC, EMEA)",2 hr 14 min,Action\n \n Adventure\n \n ...,PG-13,Walt Disney Studios Motion PicturesSee full co...,Chadwick Boseman\n \n T'Challa/Black...,Ryan Coogler\n \n Director


In [6]:
#cleaning Feature variables: Release Date
top1000["Rel_Date"] = top1000["Rel_Date"].apply(lambda x: x.split("\n", 1)[0])
top1000["Rel_Date"] = pd.to_datetime(top1000["Rel_Date"], format='%B %d, %Y')
top1000.head()

,Title,Domestic_Gr,Dom_Open,Budget,Rel_Date,Runtime,Genre,Rating,Distributor,Cast,Director
0,Star Wars: Episode VII - The Force Awakens (2015),936662225,247966675,245000000,2015-12-16,2 hr 18 min,Action\n \n Adventure\n \n ...,PG-13,Walt Disney Studios Motion PicturesSee full co...,Daisy Ridley\n \n ReySee more,J.J. Abrams\n \n Director
1,Avengers: Endgame (2019),858373000,357115007,356000000,2019-04-24,3 hr 1 min,Action\n \n Adventure\n \n ...,PG-13,Walt Disney Studios Motion PicturesSee full co...,Robert Downey Jr.\n \n Tony Stark/Ir...,Anthony Russo\n \n Director
2,Spider-Man: No Way Home (2021),804761122,260138569,NaN,2021-12-15,2 hr 28 min,Action\n \n Adventure\n \n ...,PG-13,Sony Pictures Entertainment (SPE)See full comp...,Tom Holland,Jon Watts
3,Avatar (2009),760507625,77025481,237000000,2009-12-16,2 hr 42 min,Action\n \n Adventure\n \n ...,PG-13,Twentieth Century FoxSee full company informat...,Sam Worthington\n \n Jake SullySee more,James Cameron\n \n Director
4,Black Panther (2018),700426566,202003951,NaN,2018-02-13,2 hr 14 min,Action\n \n Adventure\n \n ...,PG-13,Walt Disney Studios Motion PicturesSee full co...,Chadwick Boseman\n \n T'Challa/Black...,Ryan Coogler\n \n Director


In [7]:
top1000.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 0 to 999
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Title        1000 non-null   object        
 1   Domestic_Gr  1000 non-null   object        
 2   Dom_Open     969 non-null    object        
 3   Budget       807 non-null    object        
 4   Rel_Date     1000 non-null   datetime64[ns]
 5   Runtime      1000 non-null   object        
 6   Genre        1000 non-null   object        
 7   Rating       1000 non-null   object        
 8   Distributor  1000 non-null   object        
 9   Cast         1000 non-null   object        
 10  Director     1000 non-null   object        
dtypes: datetime64[ns](1), object(10)
memory usage: 93.8+ KB


In [8]:
#Changing Runtime to complete number of minutes
#splitting out hours first
top1000["Runtime_Hrs"] = top1000["Runtime"].apply(lambda x: x.split()[0])
top1000.head()

,Title,Domestic_Gr,Dom_Open,Budget,Rel_Date,Runtime,Genre,Rating,Distributor,Cast,Director,Runtime_Hrs
0,Star Wars: Episode VII - The Force Awakens (2015),936662225,247966675,245000000,2015-12-16,2 hr 18 min,Action\n \n Adventure\n \n ...,PG-13,Walt Disney Studios Motion PicturesSee full co...,Daisy Ridley\n \n ReySee more,J.J. Abrams\n \n Director,2
1,Avengers: Endgame (2019),858373000,357115007,356000000,2019-04-24,3 hr 1 min,Action\n \n Adventure\n \n ...,PG-13,Walt Disney Studios Motion PicturesSee full co...,Robert Downey Jr.\n \n Tony Stark/Ir...,Anthony Russo\n \n Director,3
2,Spider-Man: No Way Home (2021),804761122,260138569,NaN,2021-12-15,2 hr 28 min,Action\n \n Adventure\n \n ...,PG-13,Sony Pictures Entertainment (SPE)See full comp...,Tom Holland,Jon Watts,2
3,Avatar (2009),760507625,77025481,237000000,2009-12-16,2 hr 42 min,Action\n \n Adventure\n \n ...,PG-13,Twentieth Century FoxSee full company informat...,Sam Worthington\n \n Jake SullySee more,James Cameron\n \n Director,2
4,Black Panther (2018),700426566,202003951,NaN,2018-02-13,2 hr 14 min,Action\n \n Adventure\n \n ...,PG-13,Walt Disney Studios Motion PicturesSee full co...,Chadwick Boseman\n \n T'Challa/Black...,Ryan Coogler\n \n Director,2


In [9]:
def get_minutes(x):
    try:
        return x.split()[2]
    except:
        return None
    
top1000['Runtime_mins']=top1000.Runtime.apply(get_minutes)
top1000.head()

,Title,Domestic_Gr,Dom_Open,Budget,Rel_Date,Runtime,Genre,Rating,Distributor,Cast,Director,Runtime_Hrs,Runtime_mins
0,Star Wars: Episode VII - The Force Awakens (2015),936662225,247966675,245000000,2015-12-16,2 hr 18 min,Action\n \n Adventure\n \n ...,PG-13,Walt Disney Studios Motion PicturesSee full co...,Daisy Ridley\n \n ReySee more,J.J. Abrams\n \n Director,2,18
1,Avengers: Endgame (2019),858373000,357115007,356000000,2019-04-24,3 hr 1 min,Action\n \n Adventure\n \n ...,PG-13,Walt Disney Studios Motion PicturesSee full co...,Robert Downey Jr.\n \n Tony Stark/Ir...,Anthony Russo\n \n Director,3,1
2,Spider-Man: No Way Home (2021),804761122,260138569,NaN,2021-12-15,2 hr 28 min,Action\n \n Adventure\n \n ...,PG-13,Sony Pictures Entertainment (SPE)See full comp...,Tom Holland,Jon Watts,2,28
3,Avatar (2009),760507625,77025481,237000000,2009-12-16,2 hr 42 min,Action\n \n Adventure\n \n ...,PG-13,Twentieth Century FoxSee full company informat...,Sam Worthington\n \n Jake SullySee more,James Cameron\n \n Director,2,42
4,Black Panther (2018),700426566,202003951,NaN,2018-02-13,2 hr 14 min,Action\n \n Adventure\n \n ...,PG-13,Walt Disney Studios Motion PicturesSee full co...,Chadwick Boseman\n \n T'Challa/Black...,Ryan Coogler\n \n Director,2,14


In [10]:
top1000[top1000['Runtime_mins'].isna()].head()

,Title,Domestic_Gr,Dom_Open,Budget,Rel_Date,Runtime,Genre,Rating,Distributor,Cast,Director,Runtime_Hrs,Runtime_mins
159,Justice League (2017),229024295,93842239,NaN,2017-10-17,2 hr,Action\n \n Adventure\n \n ...,PG-13,Warner Bros.See full company information\n\n,Ben Affleck\n \n Batman/Bruce WayneS...,Zack Snyder\n \n Director,2,None
249,Mr. & Mrs. Smith (2005),186336279,50342878,110000000,2005-06-08,2 hr,Action\n \n Comedy\n \n Cr...,PG-13,Twentieth Century FoxSee full company informat...,Brad Pitt\n \n John SmithSee more,Doug Liman\n \n Director,2,None
290,Crazy Rich Asians (2018),174837452,26510140,30000000,2018-08-15,2 hr,Comedy\n \n Drama\n \n Rom...,PG-13,Warner Bros.See full company information\n\n,Constance Wu\n \n Rachel ChuSee more,Jon M. Chu\n \n Director,2,None
295,Home Alone 2: Lost in New York (1992),173585516,31126882,NaN,1992-11-20,2 hr,Adventure\n \n Comedy\n \n ...,PG,Twentieth Century FoxSee full company informat...,Macaulay Culkin\n \n KevinSee more,Chris Columbus\n \n Director,2,None
371,Mad Max: Fury Road (2015),154109060,45428128,150000000,2015-05-07,2 hr,Action\n \n Adventure\n \n ...,R,Warner Bros.See full company information\n\n,Tom Hardy\n \n Max RockatanskySee more,George Miller\n \n Director,2,None


In [11]:
top1000['Runtime_mins'] = top1000['Runtime_mins'].replace([None], np.nan)
top1000['Runtime_mins']= top1000['Runtime_mins'].fillna(0)
top1000[top1000['Runtime_mins'].isna()].head()

,Title,Domestic_Gr,Dom_Open,Budget,Rel_Date,Runtime,Genre,Rating,Distributor,Cast,Director,Runtime_Hrs,Runtime_mins


In [12]:
top1000.head()

,Title,Domestic_Gr,Dom_Open,Budget,Rel_Date,Runtime,Genre,Rating,Distributor,Cast,Director,Runtime_Hrs,Runtime_mins
0,Star Wars: Episode VII - The Force Awakens (2015),936662225,247966675,245000000,2015-12-16,2 hr 18 min,Action\n \n Adventure\n \n ...,PG-13,Walt Disney Studios Motion PicturesSee full co...,Daisy Ridley\n \n ReySee more,J.J. Abrams\n \n Director,2,18
1,Avengers: Endgame (2019),858373000,357115007,356000000,2019-04-24,3 hr 1 min,Action\n \n Adventure\n \n ...,PG-13,Walt Disney Studios Motion PicturesSee full co...,Robert Downey Jr.\n \n Tony Stark/Ir...,Anthony Russo\n \n Director,3,1
2,Spider-Man: No Way Home (2021),804761122,260138569,NaN,2021-12-15,2 hr 28 min,Action\n \n Adventure\n \n ...,PG-13,Sony Pictures Entertainment (SPE)See full comp...,Tom Holland,Jon Watts,2,28
3,Avatar (2009),760507625,77025481,237000000,2009-12-16,2 hr 42 min,Action\n \n Adventure\n \n ...,PG-13,Twentieth Century FoxSee full company informat...,Sam Worthington\n \n Jake SullySee more,James Cameron\n \n Director,2,42
4,Black Panther (2018),700426566,202003951,NaN,2018-02-13,2 hr 14 min,Action\n \n Adventure\n \n ...,PG-13,Walt Disney Studios Motion PicturesSee full co...,Chadwick Boseman\n \n T'Challa/Black...,Ryan Coogler\n \n Director,2,14


In [13]:
top1000.info()
#need to change type into float64

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 0 to 999
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Title         1000 non-null   object        
 1   Domestic_Gr   1000 non-null   object        
 2   Dom_Open      969 non-null    object        
 3   Budget        807 non-null    object        
 4   Rel_Date      1000 non-null   datetime64[ns]
 5   Runtime       1000 non-null   object        
 6   Genre         1000 non-null   object        
 7   Rating        1000 non-null   object        
 8   Distributor   1000 non-null   object        
 9   Cast          1000 non-null   object        
 10  Director      1000 non-null   object        
 11  Runtime_Hrs   1000 non-null   object        
 12  Runtime_mins  1000 non-null   object        
dtypes: datetime64[ns](1), object(12)
memory usage: 109.4+ KB


In [14]:
top1000["Runtime_Hrs"] = top1000["Runtime_Hrs"].astype(float)
top1000["Runtime_mins"] = top1000["Runtime_mins"].astype(float)
top1000.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 0 to 999
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Title         1000 non-null   object        
 1   Domestic_Gr   1000 non-null   object        
 2   Dom_Open      969 non-null    object        
 3   Budget        807 non-null    object        
 4   Rel_Date      1000 non-null   datetime64[ns]
 5   Runtime       1000 non-null   object        
 6   Genre         1000 non-null   object        
 7   Rating        1000 non-null   object        
 8   Distributor   1000 non-null   object        
 9   Cast          1000 non-null   object        
 10  Director      1000 non-null   object        
 11  Runtime_Hrs   1000 non-null   float64       
 12  Runtime_mins  1000 non-null   float64       
dtypes: datetime64[ns](1), float64(2), object(10)
memory usage: 109.4+ KB


In [15]:
top1000["Runtime_Hrs_as_Mins"] = top1000["Runtime_Hrs"]*60
top1000.head()

,Title,Domestic_Gr,Dom_Open,Budget,Rel_Date,Runtime,Genre,Rating,Distributor,Cast,Director,Runtime_Hrs,Runtime_mins,Runtime_Hrs_as_Mins
0,Star Wars: Episode VII - The Force Awakens (2015),936662225,247966675,245000000,2015-12-16,2 hr 18 min,Action\n \n Adventure\n \n ...,PG-13,Walt Disney Studios Motion PicturesSee full co...,Daisy Ridley\n \n ReySee more,J.J. Abrams\n \n Director,2.0,18.0,120.0
1,Avengers: Endgame (2019),858373000,357115007,356000000,2019-04-24,3 hr 1 min,Action\n \n Adventure\n \n ...,PG-13,Walt Disney Studios Motion PicturesSee full co...,Robert Downey Jr.\n \n Tony Stark/Ir...,Anthony Russo\n \n Director,3.0,1.0,180.0
2,Spider-Man: No Way Home (2021),804761122,260138569,NaN,2021-12-15,2 hr 28 min,Action\n \n Adventure\n \n ...,PG-13,Sony Pictures Entertainment (SPE)See full comp...,Tom Holland,Jon Watts,2.0,28.0,120.0
3,Avatar (2009),760507625,77025481,237000000,2009-12-16,2 hr 42 min,Action\n \n Adventure\n \n ...,PG-13,Twentieth Century FoxSee full company informat...,Sam Worthington\n \n Jake SullySee more,James Cameron\n \n Director,2.0,42.0,120.0
4,Black Panther (2018),700426566,202003951,NaN,2018-02-13,2 hr 14 min,Action\n \n Adventure\n \n ...,PG-13,Walt Disney Studios Motion PicturesSee full co...,Chadwick Boseman\n \n T'Challa/Black...,Ryan Coogler\n \n Director,2.0,14.0,120.0


In [16]:
top1000["Runtime_tot_mins"] = top1000["Runtime_Hrs_as_Mins"] + top1000['Runtime_mins']

In [17]:
top1000.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 0 to 999
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Title                1000 non-null   object        
 1   Domestic_Gr          1000 non-null   object        
 2   Dom_Open             969 non-null    object        
 3   Budget               807 non-null    object        
 4   Rel_Date             1000 non-null   datetime64[ns]
 5   Runtime              1000 non-null   object        
 6   Genre                1000 non-null   object        
 7   Rating               1000 non-null   object        
 8   Distributor          1000 non-null   object        
 9   Cast                 1000 non-null   object        
 10  Director             1000 non-null   object        
 11  Runtime_Hrs          1000 non-null   float64       
 12  Runtime_mins         1000 non-null   float64       
 13  Runtime_Hrs_as_Mins  1000 non-null

In [18]:
top1000.sort_values('Runtime_tot_mins', ascending=False)
#two extreme values, need to change

,Title,Domestic_Gr,Dom_Open,Budget,Rel_Date,Runtime,Genre,Rating,Distributor,Cast,Director,Runtime_Hrs,Runtime_mins,Runtime_Hrs_as_Mins,Runtime_tot_mins
833,Space Station 3D (2002),93383953,489488,NaN,2002-04-19,47 min,Documentary,R,IMAXSee full company information\n\n,Tom Cruise\n \n NarratorSee more,Toni Myers\n \n Director,47.0,0.0,2820.0,2820.0
898,Everest (1998),87178599,364244,NaN,1998-03-06,44 min,Documentary\n \n Short,PG-13,MacGillivray Freeman FilmsSee full company inf...,Liam Neeson\n \n Self - NarratorSee ...,David Breashears\n \n Director,44.0,0.0,2640.0,2640.0
219,Gone with the Wind (1939),200882193,NaN,NaN,1939-12-15,3 hr 58 min,Drama\n \n Romance\n \n War,G,Metro-Goldwyn-Mayer (MGM)See full company info...,Clark Gable\n \n Rhett Butler - Visi...,Victor Fleming\n \n Director,3.0,58.0,180.0,238.0
46,The Lord of the Rings: The Return of the King ...,378251207,72629713,94000000,2003-12-17,3 hr 21 min,Action\n \n Adventure\n \n ...,PG-13,New Line CinemaSee full company information\n\n,Elijah Wood\n \n FrodoSee more,Peter Jackson\n \n Director,3.0,21.0,180.0,201.0
640,Doctor Zhivago (1965),111721910,NaN,NaN,1965-12-22,3 hr 17 min,Drama\n \n Romance\n \n War,PG-13,Metro-Goldwyn-Mayer (MGM)See full company info...,Omar Sharif\n \n YuriSee more,David Lean\n \n Director,3.0,17.0,180.0,197.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
877,The Emperor's New Groove (2000),89636687,9812302,100000000,2000-12-15,1 hr 18 min,Adventure\n \n Animation\n \n ...,G,Walt Disney Studios Motion PicturesSee full co...,David Spade\n \n KuzcoSee more,Mark Dindal\n \n Director,1.0,18.0,60.0,78.0
895,Peter Pan (1953),87404651,NaN,4000000,1953-02-05,1 hr 17 min,Adventure\n \n Animation\n \n ...,G,Walt Disney Studios Motion PicturesSee full co...,Bobby Driscoll\n \n Peter PanSee more,Clyde Geronimi\n \n Director,1.0,17.0,60.0,77.0
828,Lady and the Tramp (1955),93602326,NaN,4000000,1955-06-22,1 hr 16 min,Adventure\n \n Animation\n \n ...,G,Walt Disney Studios Motion PicturesSee full co...,Barbara Luddy\n \n LadySee more,Clyde Geronimi\n \n Director,1.0,16.0,60.0,76.0
836,Cinderella (1950),93141149,NaN,NaN,1950-03-04,1 hr 14 min,Animation\n \n Family\n \n ...,G,Walt Disney Studios Motion PicturesSee full co...,Ilene Woods\n \n CinderellaSee more,Clyde Geronimi\n \n Director,1.0,14.0,60.0,74.0


In [19]:
top1000.loc[[833]] #runtime_tot_mins should say 47 mins

,Title,Domestic_Gr,Dom_Open,Budget,Rel_Date,Runtime,Genre,Rating,Distributor,Cast,Director,Runtime_Hrs,Runtime_mins,Runtime_Hrs_as_Mins,Runtime_tot_mins
833,Space Station 3D (2002),93383953,489488,NaN,2002-04-19,47 min,Documentary,R,IMAXSee full company information\n\n,Tom Cruise\n \n NarratorSee more,Toni Myers\n \n Director,47.0,0.0,2820.0,2820.0


In [20]:
top1000.loc[[898]] #should say 44 mins

,Title,Domestic_Gr,Dom_Open,Budget,Rel_Date,Runtime,Genre,Rating,Distributor,Cast,Director,Runtime_Hrs,Runtime_mins,Runtime_Hrs_as_Mins,Runtime_tot_mins
898,Everest (1998),87178599,364244,NaN,1998-03-06,44 min,Documentary\n \n Short,PG-13,MacGillivray Freeman FilmsSee full company inf...,Liam Neeson\n \n Self - NarratorSee ...,David Breashears\n \n Director,44.0,0.0,2640.0,2640.0


In [21]:
#changing to correct values
top1000.loc[833, ["Runtime_tot_mins"]] = 47
top1000.loc[898, ["Runtime_tot_mins"]] = 44

In [22]:
top1000.sort_values('Runtime_tot_mins', ascending=False)

,Title,Domestic_Gr,Dom_Open,Budget,Rel_Date,Runtime,Genre,Rating,Distributor,Cast,Director,Runtime_Hrs,Runtime_mins,Runtime_Hrs_as_Mins,Runtime_tot_mins
219,Gone with the Wind (1939),200882193,NaN,NaN,1939-12-15,3 hr 58 min,Drama\n \n Romance\n \n War,G,Metro-Goldwyn-Mayer (MGM)See full company info...,Clark Gable\n \n Rhett Butler - Visi...,Victor Fleming\n \n Director,3.0,58.0,180.0,238.0
46,The Lord of the Rings: The Return of the King ...,378251207,72629713,94000000,2003-12-17,3 hr 21 min,Action\n \n Adventure\n \n ...,PG-13,New Line CinemaSee full company information\n\n,Elijah Wood\n \n FrodoSee more,Peter Jackson\n \n Director,3.0,21.0,180.0,201.0
640,Doctor Zhivago (1965),111721910,NaN,NaN,1965-12-22,3 hr 17 min,Drama\n \n Romance\n \n War,PG-13,Metro-Goldwyn-Mayer (MGM)See full company info...,Omar Sharif\n \n YuriSee more,David Lean\n \n Director,3.0,17.0,180.0,197.0
798,Schindler's List (1993),96898818,656636,22000000,1993-12-15,3 hr 15 min,Biography\n \n Drama\n \n ...,R,Universal PicturesSee full company information...,Liam Neeson\n \n Oskar SchindlerSee ...,Steven Spielberg\n \n Director,3.0,15.0,180.0,195.0
6,Titanic (1997),659363944,28638131,200000000,1997-12-19,3 hr 14 min,Drama\n \n Romance,PG-13,Paramount PicturesSee full company information...,Leonardo DiCaprio\n \n Jack DawsonSe...,James Cameron\n \n Director,3.0,14.0,180.0,194.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
828,Lady and the Tramp (1955),93602326,NaN,4000000,1955-06-22,1 hr 16 min,Adventure\n \n Animation\n \n ...,G,Walt Disney Studios Motion PicturesSee full co...,Barbara Luddy\n \n LadySee more,Clyde Geronimi\n \n Director,1.0,16.0,60.0,76.0
836,Cinderella (1950),93141149,NaN,NaN,1950-03-04,1 hr 14 min,Animation\n \n Family\n \n ...,G,Walt Disney Studios Motion PicturesSee full co...,Ilene Woods\n \n CinderellaSee more,Clyde Geronimi\n \n Director,1.0,14.0,60.0,74.0
733,Bambi (1942),102247150,NaN,NaN,1942-08-21,1 hr 9 min,Adventure\n \n Animation\n \n ...,G,RKO Radio PicturesSee full company information...,Hardie Albright\n \n Adolescent Bamb...,James Algar\n \n Director,1.0,9.0,60.0,69.0
833,Space Station 3D (2002),93383953,489488,NaN,2002-04-19,47 min,Documentary,R,IMAXSee full company information\n\n,Tom Cruise\n \n NarratorSee more,Toni Myers\n \n Director,47.0,0.0,2820.0,47.0


In [23]:
top1000 = top1000.drop(columns=['Runtime_mins', 'Runtime_Hrs_as_Mins', 'Runtime_Hrs', 'Runtime'])

In [24]:
top1000["Domestic_Gr"] = top1000["Domestic_Gr"].astype(float)
top1000["Dom_Open"] = top1000["Dom_Open"].astype(float)
top1000["Budget"] = top1000["Budget"].astype(float)

In [25]:
top1000.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 0 to 999
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Title             1000 non-null   object        
 1   Domestic_Gr       1000 non-null   float64       
 2   Dom_Open          969 non-null    float64       
 3   Budget            807 non-null    float64       
 4   Rel_Date          1000 non-null   datetime64[ns]
 5   Genre             1000 non-null   object        
 6   Rating            1000 non-null   object        
 7   Distributor       1000 non-null   object        
 8   Cast              1000 non-null   object        
 9   Director          1000 non-null   object        
 10  Runtime_tot_mins  1000 non-null   float64       
dtypes: datetime64[ns](1), float64(4), object(6)
memory usage: 126.0+ KB


In [26]:
#cleaning up string features
#DISTRIBUTOR: Remove all characters after the characters 'See'
top1000['Distributor'] = top1000['Distributor'].str.split('See').str[0]
top1000.head()

,Title,Domestic_Gr,Dom_Open,Budget,Rel_Date,Genre,Rating,Distributor,Cast,Director,Runtime_tot_mins
0,Star Wars: Episode VII - The Force Awakens (2015),936662225.0,247966675.0,245000000.0,2015-12-16,Action\n \n Adventure\n \n ...,PG-13,Walt Disney Studios Motion Pictures,Daisy Ridley\n \n ReySee more,J.J. Abrams\n \n Director,138.0
1,Avengers: Endgame (2019),858373000.0,357115007.0,356000000.0,2019-04-24,Action\n \n Adventure\n \n ...,PG-13,Walt Disney Studios Motion Pictures,Robert Downey Jr.\n \n Tony Stark/Ir...,Anthony Russo\n \n Director,181.0
2,Spider-Man: No Way Home (2021),804761122.0,260138569.0,NaN,2021-12-15,Action\n \n Adventure\n \n ...,PG-13,Sony Pictures Entertainment (SPE),Tom Holland,Jon Watts,148.0
3,Avatar (2009),760507625.0,77025481.0,237000000.0,2009-12-16,Action\n \n Adventure\n \n ...,PG-13,Twentieth Century Fox,Sam Worthington\n \n Jake SullySee more,James Cameron\n \n Director,162.0
4,Black Panther (2018),700426566.0,202003951.0,NaN,2018-02-13,Action\n \n Adventure\n \n ...,PG-13,Walt Disney Studios Motion Pictures,Chadwick Boseman\n \n T'Challa/Black...,Ryan Coogler\n \n Director,134.0


In [27]:
#cleaning up string features
#CAST: Remove all characters after the characters '\n'
top1000['MainChar'] = top1000['Cast'].str.split('\n').str[0]

In [28]:
#dropping Cast variable now
top1000 = top1000.drop(columns=['Cast'])
top1000.head()

,Title,Domestic_Gr,Dom_Open,Budget,Rel_Date,Genre,Rating,Distributor,Director,Runtime_tot_mins,MainChar
0,Star Wars: Episode VII - The Force Awakens (2015),936662225.0,247966675.0,245000000.0,2015-12-16,Action\n \n Adventure\n \n ...,PG-13,Walt Disney Studios Motion Pictures,J.J. Abrams\n \n Director,138.0,Daisy Ridley
1,Avengers: Endgame (2019),858373000.0,357115007.0,356000000.0,2019-04-24,Action\n \n Adventure\n \n ...,PG-13,Walt Disney Studios Motion Pictures,Anthony Russo\n \n Director,181.0,Robert Downey Jr.
2,Spider-Man: No Way Home (2021),804761122.0,260138569.0,NaN,2021-12-15,Action\n \n Adventure\n \n ...,PG-13,Sony Pictures Entertainment (SPE),Jon Watts,148.0,Tom Holland
3,Avatar (2009),760507625.0,77025481.0,237000000.0,2009-12-16,Action\n \n Adventure\n \n ...,PG-13,Twentieth Century Fox,James Cameron\n \n Director,162.0,Sam Worthington
4,Black Panther (2018),700426566.0,202003951.0,NaN,2018-02-13,Action\n \n Adventure\n \n ...,PG-13,Walt Disney Studios Motion Pictures,Ryan Coogler\n \n Director,134.0,Chadwick Boseman


In [29]:
#cleaning up string features
#DIRECTOR: Remove all characters after the characters '\n'
top1000['Director'] = top1000['Director'].str.split('\n').str[0]
top1000.head()

,Title,Domestic_Gr,Dom_Open,Budget,Rel_Date,Genre,Rating,Distributor,Director,Runtime_tot_mins,MainChar
0,Star Wars: Episode VII - The Force Awakens (2015),936662225.0,247966675.0,245000000.0,2015-12-16,Action\n \n Adventure\n \n ...,PG-13,Walt Disney Studios Motion Pictures,J.J. Abrams,138.0,Daisy Ridley
1,Avengers: Endgame (2019),858373000.0,357115007.0,356000000.0,2019-04-24,Action\n \n Adventure\n \n ...,PG-13,Walt Disney Studios Motion Pictures,Anthony Russo,181.0,Robert Downey Jr.
2,Spider-Man: No Way Home (2021),804761122.0,260138569.0,NaN,2021-12-15,Action\n \n Adventure\n \n ...,PG-13,Sony Pictures Entertainment (SPE),Jon Watts,148.0,Tom Holland
3,Avatar (2009),760507625.0,77025481.0,237000000.0,2009-12-16,Action\n \n Adventure\n \n ...,PG-13,Twentieth Century Fox,James Cameron,162.0,Sam Worthington
4,Black Panther (2018),700426566.0,202003951.0,NaN,2018-02-13,Action\n \n Adventure\n \n ...,PG-13,Walt Disney Studios Motion Pictures,Ryan Coogler,134.0,Chadwick Boseman


In [30]:
#cleaning up string features
#GENRE: Remove all characters after the characters '\n'
top1000["Genre"] = top1000["Genre"].apply(lambda x: x.replace('\n', ", ")) 
top1000.head()

,Title,Domestic_Gr,Dom_Open,Budget,Rel_Date,Genre,Rating,Distributor,Director,Runtime_tot_mins,MainChar
0,Star Wars: Episode VII - The Force Awakens (2015),936662225.0,247966675.0,245000000.0,2015-12-16,"Action, , Adventure, , ...",PG-13,Walt Disney Studios Motion Pictures,J.J. Abrams,138.0,Daisy Ridley
1,Avengers: Endgame (2019),858373000.0,357115007.0,356000000.0,2019-04-24,"Action, , Adventure, , ...",PG-13,Walt Disney Studios Motion Pictures,Anthony Russo,181.0,Robert Downey Jr.
2,Spider-Man: No Way Home (2021),804761122.0,260138569.0,NaN,2021-12-15,"Action, , Adventure, , ...",PG-13,Sony Pictures Entertainment (SPE),Jon Watts,148.0,Tom Holland
3,Avatar (2009),760507625.0,77025481.0,237000000.0,2009-12-16,"Action, , Adventure, , ...",PG-13,Twentieth Century Fox,James Cameron,162.0,Sam Worthington
4,Black Panther (2018),700426566.0,202003951.0,NaN,2018-02-13,"Action, , Adventure, , ...",PG-13,Walt Disney Studios Motion Pictures,Ryan Coogler,134.0,Chadwick Boseman


In [31]:
#going to split out year and make an AGE variable, age since release
top1000['year'] = pd.DatetimeIndex(top1000['Rel_Date']).year
top1000.head()

,Title,Domestic_Gr,Dom_Open,Budget,Rel_Date,Genre,Rating,Distributor,Director,Runtime_tot_mins,MainChar,year
0,Star Wars: Episode VII - The Force Awakens (2015),936662225.0,247966675.0,245000000.0,2015-12-16,"Action, , Adventure, , ...",PG-13,Walt Disney Studios Motion Pictures,J.J. Abrams,138.0,Daisy Ridley,2015
1,Avengers: Endgame (2019),858373000.0,357115007.0,356000000.0,2019-04-24,"Action, , Adventure, , ...",PG-13,Walt Disney Studios Motion Pictures,Anthony Russo,181.0,Robert Downey Jr.,2019
2,Spider-Man: No Way Home (2021),804761122.0,260138569.0,NaN,2021-12-15,"Action, , Adventure, , ...",PG-13,Sony Pictures Entertainment (SPE),Jon Watts,148.0,Tom Holland,2021
3,Avatar (2009),760507625.0,77025481.0,237000000.0,2009-12-16,"Action, , Adventure, , ...",PG-13,Twentieth Century Fox,James Cameron,162.0,Sam Worthington,2009
4,Black Panther (2018),700426566.0,202003951.0,NaN,2018-02-13,"Action, , Adventure, , ...",PG-13,Walt Disney Studios Motion Pictures,Ryan Coogler,134.0,Chadwick Boseman,2018


In [32]:
top1000['Years_Since_Release'] = 2022-top1000['year']
top1000.head()

,Title,Domestic_Gr,Dom_Open,Budget,Rel_Date,Genre,Rating,Distributor,Director,Runtime_tot_mins,MainChar,year,Years_Since_Release
0,Star Wars: Episode VII - The Force Awakens (2015),936662225.0,247966675.0,245000000.0,2015-12-16,"Action, , Adventure, , ...",PG-13,Walt Disney Studios Motion Pictures,J.J. Abrams,138.0,Daisy Ridley,2015,7
1,Avengers: Endgame (2019),858373000.0,357115007.0,356000000.0,2019-04-24,"Action, , Adventure, , ...",PG-13,Walt Disney Studios Motion Pictures,Anthony Russo,181.0,Robert Downey Jr.,2019,3
2,Spider-Man: No Way Home (2021),804761122.0,260138569.0,NaN,2021-12-15,"Action, , Adventure, , ...",PG-13,Sony Pictures Entertainment (SPE),Jon Watts,148.0,Tom Holland,2021,1
3,Avatar (2009),760507625.0,77025481.0,237000000.0,2009-12-16,"Action, , Adventure, , ...",PG-13,Twentieth Century Fox,James Cameron,162.0,Sam Worthington,2009,13
4,Black Panther (2018),700426566.0,202003951.0,NaN,2018-02-13,"Action, , Adventure, , ...",PG-13,Walt Disney Studios Motion Pictures,Ryan Coogler,134.0,Chadwick Boseman,2018,4


In [33]:
#going to split out MONTH and assess if any relationship with MONTH of movie RELEASE
top1000['Month_of_Release'] = pd.DatetimeIndex(top1000['Rel_Date']).month
top1000.head()

,Title,Domestic_Gr,Dom_Open,Budget,Rel_Date,Genre,Rating,Distributor,Director,Runtime_tot_mins,MainChar,year,Years_Since_Release,Month_of_Release
0,Star Wars: Episode VII - The Force Awakens (2015),936662225.0,247966675.0,245000000.0,2015-12-16,"Action, , Adventure, , ...",PG-13,Walt Disney Studios Motion Pictures,J.J. Abrams,138.0,Daisy Ridley,2015,7,12
1,Avengers: Endgame (2019),858373000.0,357115007.0,356000000.0,2019-04-24,"Action, , Adventure, , ...",PG-13,Walt Disney Studios Motion Pictures,Anthony Russo,181.0,Robert Downey Jr.,2019,3,4
2,Spider-Man: No Way Home (2021),804761122.0,260138569.0,NaN,2021-12-15,"Action, , Adventure, , ...",PG-13,Sony Pictures Entertainment (SPE),Jon Watts,148.0,Tom Holland,2021,1,12
3,Avatar (2009),760507625.0,77025481.0,237000000.0,2009-12-16,"Action, , Adventure, , ...",PG-13,Twentieth Century Fox,James Cameron,162.0,Sam Worthington,2009,13,12
4,Black Panther (2018),700426566.0,202003951.0,NaN,2018-02-13,"Action, , Adventure, , ...",PG-13,Walt Disney Studios Motion Pictures,Ryan Coogler,134.0,Chadwick Boseman,2018,4,2


In [34]:
top1000['Month_of_Release'].value_counts()
#most movies were release in 5-7 and 11 and 12, going to dummy code ALL months though

12    156
6     152
7     117
11    114
5     109
3      75
8      59
10     55
2      53
9      45
4      40
1      25
Name: Month_of_Release, dtype: int64

In [35]:
months = {1: 'Jan', 2: 'Feb', 3: 'Mar', 4:'Apr', 5:'May', 6:'Jun', 7:'Jul',
         8:'Aug', 9:'Sep', 10:'Oct', 11:'Nov', 12:'Dec'}

In [36]:
top1000 = top1000.replace({"Month_of_Release": months})
top1000.head()

,Title,Domestic_Gr,Dom_Open,Budget,Rel_Date,Genre,Rating,Distributor,Director,Runtime_tot_mins,MainChar,year,Years_Since_Release,Month_of_Release
0,Star Wars: Episode VII - The Force Awakens (2015),936662225.0,247966675.0,245000000.0,2015-12-16,"Action, , Adventure, , ...",PG-13,Walt Disney Studios Motion Pictures,J.J. Abrams,138.0,Daisy Ridley,2015,7,Dec
1,Avengers: Endgame (2019),858373000.0,357115007.0,356000000.0,2019-04-24,"Action, , Adventure, , ...",PG-13,Walt Disney Studios Motion Pictures,Anthony Russo,181.0,Robert Downey Jr.,2019,3,Apr
2,Spider-Man: No Way Home (2021),804761122.0,260138569.0,NaN,2021-12-15,"Action, , Adventure, , ...",PG-13,Sony Pictures Entertainment (SPE),Jon Watts,148.0,Tom Holland,2021,1,Dec
3,Avatar (2009),760507625.0,77025481.0,237000000.0,2009-12-16,"Action, , Adventure, , ...",PG-13,Twentieth Century Fox,James Cameron,162.0,Sam Worthington,2009,13,Dec
4,Black Panther (2018),700426566.0,202003951.0,NaN,2018-02-13,"Action, , Adventure, , ...",PG-13,Walt Disney Studios Motion Pictures,Ryan Coogler,134.0,Chadwick Boseman,2018,4,Feb


In [37]:
top1000['Rating'].value_counts()
#going to lump Approved

PG-13       441
PG          260
R           244
G            54
Approved      1
Name: Rating, dtype: int64

In [38]:
top1000.loc[top1000['Rating'] == 'Approved']
#according to IMDB, this movie is rated G, going to change

,Title,Domestic_Gr,Dom_Open,Budget,Rel_Date,Genre,Rating,Distributor,Director,Runtime_tot_mins,MainChar,year,Years_Since_Release,Month_of_Release
250,Snow White and the Seven Dwarfs (1937),184925486.0,NaN,1499000.0,1937-12-21,"Adventure, , Animation, , ...",Approved,Walt Disney Studios Motion Pictures,William Cottrell,83.0,Adriana Caselotti,1937,85,Dec


In [39]:
top1000['Rating'] = top1000['Rating'].replace(['Approved'],'G')
top1000.loc[top1000['Rating'] == 'Approved']
#cleared

,Title,Domestic_Gr,Dom_Open,Budget,Rel_Date,Genre,Rating,Distributor,Director,Runtime_tot_mins,MainChar,year,Years_Since_Release,Month_of_Release


In [40]:
top1000.columns

Index(['Title', 'Domestic_Gr', 'Dom_Open', 'Budget', 'Rel_Date', 'Genre',
       'Rating', 'Distributor', 'Director', 'Runtime_tot_mins', 'MainChar',
       'year', 'Years_Since_Release', 'Month_of_Release'],
      dtype='object')

In [41]:
top1000.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 0 to 999
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Title                1000 non-null   object        
 1   Domestic_Gr          1000 non-null   float64       
 2   Dom_Open             969 non-null    float64       
 3   Budget               807 non-null    float64       
 4   Rel_Date             1000 non-null   datetime64[ns]
 5   Genre                1000 non-null   object        
 6   Rating               1000 non-null   object        
 7   Distributor          1000 non-null   object        
 8   Director             1000 non-null   object        
 9   Runtime_tot_mins     1000 non-null   float64       
 10  MainChar             1000 non-null   object        
 11  year                 1000 non-null   int64         
 12  Years_Since_Release  1000 non-null   int64         
 13  Month_of_Release     1000 non-null

In [42]:
#missing values for Dom_Open, Budget, Rating, and Distributor
top1000[top1000['Distributor'].isna()].head()
#according to Wikipedia, the distributor is Universal Pictures

,Title,Domestic_Gr,Dom_Open,Budget,Rel_Date,Genre,Rating,Distributor,Director,Runtime_tot_mins,MainChar,year,Years_Since_Release,Month_of_Release


In [43]:
top1000.loc[286, ["Distributor"]] = "Universal Pictures"

In [44]:
#124 rows of missing Rating value
top1000[top1000['Rating'].isna()].head(124)

,Title,Domestic_Gr,Dom_Open,Budget,Rel_Date,Genre,Rating,Distributor,Director,Runtime_tot_mins,MainChar,year,Years_Since_Release,Month_of_Release


In [45]:
top1000.head()

,Title,Domestic_Gr,Dom_Open,Budget,Rel_Date,Genre,Rating,Distributor,Director,Runtime_tot_mins,MainChar,year,Years_Since_Release,Month_of_Release
0,Star Wars: Episode VII - The Force Awakens (2015),936662225.0,247966675.0,245000000.0,2015-12-16,"Action, , Adventure, , ...",PG-13,Walt Disney Studios Motion Pictures,J.J. Abrams,138.0,Daisy Ridley,2015,7,Dec
1,Avengers: Endgame (2019),858373000.0,357115007.0,356000000.0,2019-04-24,"Action, , Adventure, , ...",PG-13,Walt Disney Studios Motion Pictures,Anthony Russo,181.0,Robert Downey Jr.,2019,3,Apr
2,Spider-Man: No Way Home (2021),804761122.0,260138569.0,NaN,2021-12-15,"Action, , Adventure, , ...",PG-13,Sony Pictures Entertainment (SPE),Jon Watts,148.0,Tom Holland,2021,1,Dec
3,Avatar (2009),760507625.0,77025481.0,237000000.0,2009-12-16,"Action, , Adventure, , ...",PG-13,Twentieth Century Fox,James Cameron,162.0,Sam Worthington,2009,13,Dec
4,Black Panther (2018),700426566.0,202003951.0,NaN,2018-02-13,"Action, , Adventure, , ...",PG-13,Walt Disney Studios Motion Pictures,Ryan Coogler,134.0,Chadwick Boseman,2018,4,Feb


In [46]:
top1000.to_pickle("top1000_clean.pkl")
top1000_clean = pd.read_pickle("top1000_clean.pkl")
top1000_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 0 to 999
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Title                1000 non-null   object        
 1   Domestic_Gr          1000 non-null   float64       
 2   Dom_Open             969 non-null    float64       
 3   Budget               807 non-null    float64       
 4   Rel_Date             1000 non-null   datetime64[ns]
 5   Genre                1000 non-null   object        
 6   Rating               1000 non-null   object        
 7   Distributor          1000 non-null   object        
 8   Director             1000 non-null   object        
 9   Runtime_tot_mins     1000 non-null   float64       
 10  MainChar             1000 non-null   object        
 11  year                 1000 non-null   int64         
 12  Years_Since_Release  1000 non-null   int64         
 13  Month_of_Release     1000 non-null

In [47]:
#Let's merge in the OSCARs data
oscars = pd.read_pickle("oscars_final.pkl")
oscars = oscars.rename(columns={'year_film':'year'})
oscars.head()

,year,category,name,film,Person_freq,cumulative_freq
0,1927,ACTOR,Richard Barthelmess,The Noose,1,1
1,1927,ACTOR,Emil Jannings,The Last Command,1,1
2,1927,ACTRESS,Louise Dresser,A Ship Comes In,1,1
3,1927,ACTRESS,Janet Gaynor,7th Heaven,1,1
4,1927,ACTRESS,Gloria Swanson,Sadie Thompson,1,1


In [48]:
top1000_clean = pd.read_pickle("top1000_clean.pkl")
top1000_clean.head()

,Title,Domestic_Gr,Dom_Open,Budget,Rel_Date,Genre,Rating,Distributor,Director,Runtime_tot_mins,MainChar,year,Years_Since_Release,Month_of_Release
0,Star Wars: Episode VII - The Force Awakens (2015),936662225.0,247966675.0,245000000.0,2015-12-16,"Action, , Adventure, , ...",PG-13,Walt Disney Studios Motion Pictures,J.J. Abrams,138.0,Daisy Ridley,2015,7,Dec
1,Avengers: Endgame (2019),858373000.0,357115007.0,356000000.0,2019-04-24,"Action, , Adventure, , ...",PG-13,Walt Disney Studios Motion Pictures,Anthony Russo,181.0,Robert Downey Jr.,2019,3,Apr
2,Spider-Man: No Way Home (2021),804761122.0,260138569.0,NaN,2021-12-15,"Action, , Adventure, , ...",PG-13,Sony Pictures Entertainment (SPE),Jon Watts,148.0,Tom Holland,2021,1,Dec
3,Avatar (2009),760507625.0,77025481.0,237000000.0,2009-12-16,"Action, , Adventure, , ...",PG-13,Twentieth Century Fox,James Cameron,162.0,Sam Worthington,2009,13,Dec
4,Black Panther (2018),700426566.0,202003951.0,NaN,2018-02-13,"Action, , Adventure, , ...",PG-13,Walt Disney Studios Motion Pictures,Ryan Coogler,134.0,Chadwick Boseman,2018,4,Feb


In [49]:
#going to merge them based on year, director and/or mainchar
#need to do a left or right join because want to retain ALL rows in top1000_clean df.
#left_merged = pd.merge(top1000_clean, oscars, how='left',
                      #on=['year', 'name'])
#left_merged.head()

top_1000_clean_director = top1000_clean.merge(oscars, how = 'left', right_on=['year','name'], 
                                               left_on=['year','Director']) 

In [50]:
#top_1000_clean_director.head()
top_1000_clean_director = top_1000_clean_director.rename(columns={'cumulative_freq':'cumul_freq_direct'})
top_1000_clean_director.head()


,Title,Domestic_Gr,Dom_Open,Budget,Rel_Date,Genre,Rating,Distributor,Director,Runtime_tot_mins,MainChar,year,Years_Since_Release,Month_of_Release,category,name,film,Person_freq,cumul_freq_direct
0,Star Wars: Episode VII - The Force Awakens (2015),936662225.0,247966675.0,245000000.0,2015-12-16,"Action, , Adventure, , ...",PG-13,Walt Disney Studios Motion Pictures,J.J. Abrams,138.0,Daisy Ridley,2015,7,Dec,NaN,NaN,NaN,NaN,NaN
1,Avengers: Endgame (2019),858373000.0,357115007.0,356000000.0,2019-04-24,"Action, , Adventure, , ...",PG-13,Walt Disney Studios Motion Pictures,Anthony Russo,181.0,Robert Downey Jr.,2019,3,Apr,NaN,NaN,NaN,NaN,NaN
2,Spider-Man: No Way Home (2021),804761122.0,260138569.0,NaN,2021-12-15,"Action, , Adventure, , ...",PG-13,Sony Pictures Entertainment (SPE),Jon Watts,148.0,Tom Holland,2021,1,Dec,NaN,NaN,NaN,NaN,NaN
3,Avatar (2009),760507625.0,77025481.0,237000000.0,2009-12-16,"Action, , Adventure, , ...",PG-13,Twentieth Century Fox,James Cameron,162.0,Sam Worthington,2009,13,Dec,DIRECTING,James Cameron,Avatar,1.0,2.0
4,Black Panther (2018),700426566.0,202003951.0,NaN,2018-02-13,"Action, , Adventure, , ...",PG-13,Walt Disney Studios Motion Pictures,Ryan Coogler,134.0,Chadwick Boseman,2018,4,Feb,NaN,NaN,NaN,NaN,NaN


In [51]:
top_1000_clean_actor = top_1000_clean_director.merge(oscars, how = 'left', right_on=['year','name'], 
                                               left_on=['year','MainChar']) 
top_1000_clean_actor = top_1000_clean_actor.rename(columns={'cumulative_freq':'cumul_freq_actor'})
top_1000_clean_actor.head()

,Title,Domestic_Gr,Dom_Open,Budget,Rel_Date,Genre,Rating,Distributor,Director,Runtime_tot_mins,...,category_x,name_x,film_x,Person_freq_x,cumul_freq_direct,category_y,name_y,film_y,Person_freq_y,cumul_freq_actor
0,Star Wars: Episode VII - The Force Awakens (2015),936662225.0,247966675.0,245000000.0,2015-12-16,"Action, , Adventure, , ...",PG-13,Walt Disney Studios Motion Pictures,J.J. Abrams,138.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Avengers: Endgame (2019),858373000.0,357115007.0,356000000.0,2019-04-24,"Action, , Adventure, , ...",PG-13,Walt Disney Studios Motion Pictures,Anthony Russo,181.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Spider-Man: No Way Home (2021),804761122.0,260138569.0,NaN,2021-12-15,"Action, , Adventure, , ...",PG-13,Sony Pictures Entertainment (SPE),Jon Watts,148.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Avatar (2009),760507625.0,77025481.0,237000000.0,2009-12-16,"Action, , Adventure, , ...",PG-13,Twentieth Century Fox,James Cameron,162.0,...,DIRECTING,James Cameron,Avatar,1.0,2.0,NaN,NaN,NaN,NaN,NaN
4,Black Panther (2018),700426566.0,202003951.0,NaN,2018-02-13,"Action, , Adventure, , ...",PG-13,Walt Disney Studios Motion Pictures,Ryan Coogler,134.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [52]:
#going to drop unnecessary variables
top1000 = top_1000_clean_actor.drop(columns=['Person_freq_x', 'Person_freq_y'])

In [53]:
top1000.info()
#fill in 0 for those without any merged data
#but first let's check if some did not merge due to messy strings

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1009 entries, 0 to 1008
Data columns (total 22 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Title                1009 non-null   object        
 1   Domestic_Gr          1009 non-null   float64       
 2   Dom_Open             978 non-null    float64       
 3   Budget               812 non-null    float64       
 4   Rel_Date             1009 non-null   datetime64[ns]
 5   Genre                1009 non-null   object        
 6   Rating               1009 non-null   object        
 7   Distributor          1009 non-null   object        
 8   Director             1009 non-null   object        
 9   Runtime_tot_mins     1009 non-null   float64       
 10  MainChar             1009 non-null   object        
 11  year                 1009 non-null   int64         
 12  Years_Since_Release  1009 non-null   int64         
 13  Month_of_Release     1009 non-nul

In [54]:
# filling in missing values with 0
top1000['cumul_freq_direct_r'] = top1000['cumul_freq_direct'].fillna(0)
top1000['cumul_freq_actor_r'] = top1000['cumul_freq_actor'].fillna(0)

In [55]:
top1000.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1009 entries, 0 to 1008
Data columns (total 24 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Title                1009 non-null   object        
 1   Domestic_Gr          1009 non-null   float64       
 2   Dom_Open             978 non-null    float64       
 3   Budget               812 non-null    float64       
 4   Rel_Date             1009 non-null   datetime64[ns]
 5   Genre                1009 non-null   object        
 6   Rating               1009 non-null   object        
 7   Distributor          1009 non-null   object        
 8   Director             1009 non-null   object        
 9   Runtime_tot_mins     1009 non-null   float64       
 10  MainChar             1009 non-null   object        
 11  year                 1009 non-null   int64         
 12  Years_Since_Release  1009 non-null   int64         
 13  Month_of_Release     1009 non-nul

In [56]:
#going to drop unnecessary variables
top1000_oscars = top1000.drop(columns=['category_x', 'name_x',
                                     'film_x', 'cumul_freq_direct', 'category_y',
                                     'name_y', 'film_y', 'cumul_freq_actor'])

In [57]:
#creating STAR POWER variable
top1000_oscars['star_power'] = top1000_oscars['cumul_freq_direct_r'] + top1000_oscars['cumul_freq_actor_r']


In [58]:
top1000_oscars.head()

,Title,Domestic_Gr,Dom_Open,Budget,Rel_Date,Genre,Rating,Distributor,Director,Runtime_tot_mins,MainChar,year,Years_Since_Release,Month_of_Release,cumul_freq_direct_r,cumul_freq_actor_r,star_power
0,Star Wars: Episode VII - The Force Awakens (2015),936662225.0,247966675.0,245000000.0,2015-12-16,"Action, , Adventure, , ...",PG-13,Walt Disney Studios Motion Pictures,J.J. Abrams,138.0,Daisy Ridley,2015,7,Dec,0.0,0.0,0.0
1,Avengers: Endgame (2019),858373000.0,357115007.0,356000000.0,2019-04-24,"Action, , Adventure, , ...",PG-13,Walt Disney Studios Motion Pictures,Anthony Russo,181.0,Robert Downey Jr.,2019,3,Apr,0.0,0.0,0.0
2,Spider-Man: No Way Home (2021),804761122.0,260138569.0,NaN,2021-12-15,"Action, , Adventure, , ...",PG-13,Sony Pictures Entertainment (SPE),Jon Watts,148.0,Tom Holland,2021,1,Dec,0.0,0.0,0.0
3,Avatar (2009),760507625.0,77025481.0,237000000.0,2009-12-16,"Action, , Adventure, , ...",PG-13,Twentieth Century Fox,James Cameron,162.0,Sam Worthington,2009,13,Dec,2.0,0.0,2.0
4,Black Panther (2018),700426566.0,202003951.0,NaN,2018-02-13,"Action, , Adventure, , ...",PG-13,Walt Disney Studios Motion Pictures,Ryan Coogler,134.0,Chadwick Boseman,2018,4,Feb,0.0,0.0,0.0


In [59]:
top1000_oscars.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1009 entries, 0 to 1008
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Title                1009 non-null   object        
 1   Domestic_Gr          1009 non-null   float64       
 2   Dom_Open             978 non-null    float64       
 3   Budget               812 non-null    float64       
 4   Rel_Date             1009 non-null   datetime64[ns]
 5   Genre                1009 non-null   object        
 6   Rating               1009 non-null   object        
 7   Distributor          1009 non-null   object        
 8   Director             1009 non-null   object        
 9   Runtime_tot_mins     1009 non-null   float64       
 10  MainChar             1009 non-null   object        
 11  year                 1009 non-null   int64         
 12  Years_Since_Release  1009 non-null   int64         
 13  Month_of_Release     1009 non-nul

In [60]:
top1000_oscars.to_pickle("top1000_oscars.pkl")
top1000_oscars = pd.read_pickle("top1000_oscars.pkl")
top1000_oscars.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1009 entries, 0 to 1008
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Title                1009 non-null   object        
 1   Domestic_Gr          1009 non-null   float64       
 2   Dom_Open             978 non-null    float64       
 3   Budget               812 non-null    float64       
 4   Rel_Date             1009 non-null   datetime64[ns]
 5   Genre                1009 non-null   object        
 6   Rating               1009 non-null   object        
 7   Distributor          1009 non-null   object        
 8   Director             1009 non-null   object        
 9   Runtime_tot_mins     1009 non-null   float64       
 10  MainChar             1009 non-null   object        
 11  year                 1009 non-null   int64         
 12  Years_Since_Release  1009 non-null   int64         
 13  Month_of_Release     1009 non-nul